In [46]:
from utils.database_connection import DatabaseConnection
import utils.results_repository as rep
import pandas as pd
import numpy as np
from utils.column_name_utils import *
import utils.query_utils as query
import matplotlib.pyplot as plot
import utils.dataframe_utils as df_util
import utils.path_utils as path_util
import utils.latex_utils as tex
import pandas as pd
from scipy.stats import shapiro, mannwhitneyu

In [6]:
db = DatabaseConnection()

Connected to MySQL Server version  8.0.31-23


In [7]:
measurements_used = 30
energy_per_time = 1000

In [8]:
experiment_name = "same-cpp-multi-core-micro-benchmark-different-compiler"
experiment_number = 1

used_dut_name = [
    'workstationone'
]

used_dut_name.sort()

used_dut_env = 'prod'

windows = "win32nt"
linux = "unix"

used_os_names = [
    windows, linux
]

used_test_case_names = [
    'fannkuch-redux.exe',
    'mandelbrot.exe'
]

used_test_case_names.sort()

used_compilers = [
    'clang', 'intel-one-api', 'min-gw', 'msvc'
]

used_compilers.sort()

used_thread = [
    'multi-threaded'
]

used_thread.sort()

used_test_case_name = 'idle-case.exe'

ipg = "ipg"
lhm = "lhm"
rapl = "rapl"
clamp = "clamp"
plug = "plug"

windows_mi = [
    ipg, lhm, clamp, plug
]

linux_mi = [
    rapl, clamp, plug
]

used_mi_names = [
    lhm
]

used_mi_names.sort()

ipg_sample_rate = 333
lhm_sample_rate = 100

used_min_temp = 0
used_max_temp = 200
used_burn_in = 1
used_allocated_cores = [
    []
]

used_allocated_cores.sort()

In [9]:
used_dut_name_str = "','".join(used_dut_name)
used_os_name_str = "','".join(used_os_names)

In [10]:
dut = rep.get_devices_under_test(db)

dut

,Id,Name,Os,Env
0,1,workstationone,win32nt,prod
1,2,workstationone,unix,prod
2,3,workstationone,unix,dev
3,4,workstationone,win32nt,dev


In [11]:
used_dut = dut.query(f"{ENV} == '{used_dut_env}' and {NAME} in ('{used_dut_name_str}') and Os in ('{used_os_name_str}')")

used_dut

,Id,Name,Os,Env
0,1,workstationone,win32nt,prod
1,2,workstationone,unix,prod


In [12]:
tc = rep.get_test_cases(db)

tc

,Id,Name,Compiler,Language,Optimizations,TestCaseSize,Parameter,Threads
0,1,fannkuch-redux.exe,clang,c++,-openmp,microbenchmarks,12,multi-threaded
1,2,mandelbrot.exe,clang,c++,-openmp,microbenchmarks,16000,multi-threaded
2,3,idle-case.exe,clang,c++,-openmp,microbenchmarks,,multi-threaded
3,4,fannkuch-redux.exe,intel-one-api,c++,-openmp,microbenchmarks,12,multi-threaded
4,5,mandelbrot.exe,intel-one-api,c++,-openmp,microbenchmarks,16000,multi-threaded
5,6,idle-case.exe,intel-one-api,c++,-openmp,microbenchmarks,,multi-threaded
6,7,fannkuch-redux.exe,min-gw,c++,-openmp,microbenchmarks,12,multi-threaded
7,8,mandelbrot.exe,min-gw,c++,-openmp,microbenchmarks,16000,multi-threaded
8,9,idle-case.exe,min-gw,c++,-openmp,microbenchmarks,,multi-threaded
9,10,fannkuch-redux.exe,msvc,c++,-openmp,microbenchmarks,12,multi-threaded


In [13]:
used_test_case_name_str = "','".join(used_test_case_names)
used_compiler_str = "','".join(used_compilers)
used_thread_str = "','".join(used_thread)

used_tc = tc.query(f"{COMPILER} in ('{used_compiler_str}') and {NAME} in ('{used_test_case_name_str}') and {THREADS} in ('{used_thread_str}')")

In [14]:
used_idle_case = tc.query(f"{COMPILER} in ('{used_compiler_str}') and {NAME} == '{used_test_case_name}' and {THREADS} in ('{used_thread_str}')")

In [15]:
mi = rep.get_measuring_instruments(db)

mi

,Id,Name,SampleRate
0,1,ipg,100
1,2,lhm,100
2,3,clamp,100
3,4,rapl,100
4,5,clamp,500
5,6,scaphandre,100


In [16]:
used_mi_query_list = []

if ipg in used_mi_names:
    used_mi_query_list.append(f"({NAME} == '{ipg}' and {SAMPLE_RATE} == {ipg_sample_rate})")

if lhm in used_mi_names:
    used_mi_query_list.append(f"({NAME} == '{lhm}' and {SAMPLE_RATE} == {lhm_sample_rate})")

used_mi = mi.query(" OR ".join(used_mi_query_list))

In [17]:
used_allocated_cores_str = "','".join([str(x) for x in used_allocated_cores])

In [18]:
config = rep.get_configurations(db)

config

,Id,MinTemp,MaxTemp,Burnin,AllocatedCores
0,1,0,200,1,[]
1,2,0,200,0,[1]
2,3,0,200,0,"[0, 1, 2, 3]"


In [19]:
used_configs = config.query(f"{MIN_TEMP} == {used_min_temp} and " +
                            f"{MAX_TEMP} == {used_max_temp} and " +
                            f"{BURN_IN} == {used_burn_in} and " +
                            f"{ALLOCATED_CORES} in ('{used_allocated_cores_str}')")

In [20]:
exp = rep.get_distinct_experiments(db)

exp

,Experiment,Name
0,1,same-cpp-multi-core-micro-benchmark-different-...
1,1,test


In [21]:
used_dut

,Id,Name,Os,Env
0,1,workstationone,win32nt,prod
1,2,workstationone,unix,prod


In [22]:
used_tc

,Id,Name,Compiler,Language,Optimizations,TestCaseSize,Parameter,Threads
0,1,fannkuch-redux.exe,clang,c++,-openmp,microbenchmarks,12,multi-threaded
1,2,mandelbrot.exe,clang,c++,-openmp,microbenchmarks,16000,multi-threaded
3,4,fannkuch-redux.exe,intel-one-api,c++,-openmp,microbenchmarks,12,multi-threaded
4,5,mandelbrot.exe,intel-one-api,c++,-openmp,microbenchmarks,16000,multi-threaded
6,7,fannkuch-redux.exe,min-gw,c++,-openmp,microbenchmarks,12,multi-threaded
7,8,mandelbrot.exe,min-gw,c++,-openmp,microbenchmarks,16000,multi-threaded
9,10,fannkuch-redux.exe,msvc,c++,-openmp,microbenchmarks,12,multi-threaded
10,11,mandelbrot.exe,msvc,c++,-openmp,microbenchmarks,16000,multi-threaded


In [23]:
used_mi

,Id,Name,SampleRate
1,2,lhm,100


In [24]:
used_idle_case

,Id,Name,Compiler,Language,Optimizations,TestCaseSize,Parameter,Threads
2,3,idle-case.exe,clang,c++,-openmp,microbenchmarks,,multi-threaded
5,6,idle-case.exe,intel-one-api,c++,-openmp,microbenchmarks,,multi-threaded
8,9,idle-case.exe,min-gw,c++,-openmp,microbenchmarks,,multi-threaded
11,12,idle-case.exe,msvc,c++,-openmp,microbenchmarks,,multi-threaded


In [25]:
used_configs

,Id,MinTemp,MaxTemp,Burnin,AllocatedCores
0,1,0,200,1,[]


In [26]:
cpu_energy_results = {}
dram_energy_results = {}
gpu_energy_results = {}
duration_results = {}

cpu_dynamic_energy_consumption = {}
gpu_dynamic_energy_consumption = {}
dram_dynamic_energy_consumption = {}

for dut_index, dut_row in used_dut.iterrows():
    dut_id = dut_row[ID]
    dut_name = dut_row[NAME]
    dut_os = dut_row[OS]
    for mi_index, mi_row in used_mi.iterrows():
        mi_id = mi_row[ID]
        mi_name = mi_row[NAME]
        
        if dut_os == windows and not mi_name in windows_mi:
            continue
        
        if dut_os == linux and not mi_name in linux_mi:
            continue
        
        for config_index, config_row in used_configs.iterrows():
            config_id = config_row[ID]
            config_cores = config_row[ALLOCATED_CORES]
            if config_cores == "[]":
                config_cores = 'all'
            for tc_index, tc_row in used_tc.iterrows():
                tc_id = tc_row[ID]
                tc_name = tc_row[NAME]
                tc_compiler = tc_row[COMPILER]
                tc_language = tc_row[LANGUAGE]
                tc_optimizations = tc_row[OPTIMIZATIONS]
                tc_test_case_size = tc_row[TEST_CASE_SIZE]
                tc_threads = tc_row[THREADS]
                
                tc_idle_case = query.get_idle_case(used_idle_case, tc_compiler, tc_language, tc_optimizations, tc_test_case_size, tc_threads)
                tc_idle_case_id =int(tc_idle_case[ID].iloc[0])
                
                tc_collection = rep.get_collections(db, dut_id, mi_id, config_id, tc_id, experiment_name, experiment_number)
                tc_idle_case_collection = rep.get_collections(db, dut_id, mi_id, config_id, tc_idle_case_id, experiment_name, experiment_number)
                
                tc_collection_id = int(tc_collection[ID].iloc[0])
                tc_idle_case_collection_id = int(tc_idle_case_collection[ID].iloc[0])
                
                tc_measurements = rep.get_measurements(db, tc_collection_id)
                tc_idle_case_measurements = rep.get_measurements(db, tc_idle_case_collection_id)
                
                key = f"{tc_name}.{mi_name}.{dut_name}.{config_cores}.{tc_compiler}.{dut_os}"
                
                dram_energy_results[key] = list(tc_measurements["DramEnergyInJoules"])
                cpu_energy_results[key] = list(tc_measurements["CpuEnergyInJoules"])
                gpu_energy_results[key] = list(tc_measurements["GpuEnergyInJoules"])
                duration_results[key] = list(tc_measurements["Duration"])
                
                idle_cpu_consumption = df_util.get_cpu_energy_per_time(tc_idle_case_measurements, energy_per_time)
                idle_gpu_consumption = df_util.get_gpu_energy_per_time(tc_idle_case_measurements, energy_per_time)
                idle_dram_consumption = df_util.get_dram_energy_per_time(tc_idle_case_measurements, energy_per_time)
                
                cpu_dynamic_energy_consumption[key] = df_util.get_cpu_dynamic_energy(tc_measurements, idle_cpu_consumption, energy_per_time)
                gpu_dynamic_energy_consumption[key] = df_util.get_gpu_dynamic_energy(tc_measurements, idle_gpu_consumption, energy_per_time)
                dram_dynamic_energy_consumption[key] = df_util.get_dram_dynamic_energy(tc_measurements, idle_dram_consumption, energy_per_time)

In [27]:
cpu_energy_results

{'fannkuch-redux.exe.lhm.workstationone.all.clang.win32nt': [1304.7605146294895,
  1342.1499787654875,
  1315.0662422698458,
  1312.4645667145292,
  1415.9192900236098,
  1314.3786422109283,
  1323.193174896604,
  1356.6118318194356,
  1322.4708447881846,
  1320.3146268196172,
  1377.1644980307724,
  1306.6409855582292,
  1330.5353170676,
  1355.2685347196286,
  1314.6042494414758,
  1316.946376838873,
  1367.8991972629735,
  1345.2434068092286,
  1312.7882007070689,
  1384.9104901639878,
  1339.7718174236186,
  1329.3607494089338,
  1351.509269832538,
  1313.6435466686657,
  1316.7379436029307,
  1318.878676751172,
  1315.4524155479405,
  1309.197878929561,
  1318.6168310475991,
  1378.0022954889932],
 'mandelbrot.exe.lhm.workstationone.all.clang.win32nt': [1466.2226863680776,
  1471.3105444879006,
  1472.7624026790618,
  1475.716310308512,
  1466.0254991305453,
  1470.6135199121127,
  1471.0596576066664,
  1469.5375757965874,
  1466.7581204286391,
  1469.9324990488565,
  1468.6284289

In [28]:
config_cores == "[]"
for key, value in dram_energy_results.items():
    dram_energy_results[key] = value[:measurements_used]
    
for key, value in cpu_energy_results.items():
    cpu_energy_results[key] = value[:measurements_used]

for key, value in gpu_energy_results.items():
    gpu_energy_results[key] = value[:measurements_used]

for key, value in duration_results.items():
    duration_results[key] = value[:measurements_used]
    
for key, value in cpu_dynamic_energy_consumption.items():
    cpu_dynamic_energy_consumption[key] = value[:measurements_used]

for key, value in gpu_dynamic_energy_consumption.items():
    gpu_dynamic_energy_consumption[key] = value[:measurements_used]
    
for key, value in dram_dynamic_energy_consumption.items():
    dram_dynamic_energy_consumption[key] = value[:measurements_used]

In [29]:
## Energy consumption for the entire test case
dram_df = pd.DataFrame(dram_energy_results)
dram_df =dram_df.reindex(columns=sorted(dram_df.columns))

cpu_df = pd.DataFrame(cpu_energy_results)
cpu_df = cpu_df.reindex(columns=sorted(cpu_df.columns))

gpu_df = pd.DataFrame(gpu_energy_results)
gpu_df = gpu_df.reindex(columns=sorted(gpu_df.columns))

duration_df = pd.DataFrame(duration_results)
duration_df = duration_df.reindex(columns=sorted(duration_df.columns))

## Dynamic energy data frames
dram_dynamic_df = pd.DataFrame(dram_dynamic_energy_consumption)
dram_dynamic_df = dram_dynamic_df.reindex(columns=sorted(duration_df.columns))

cpu_dynamic_df = pd.DataFrame(cpu_dynamic_energy_consumption)
cpu_dynamic_df = cpu_dynamic_df.reindex(columns=sorted(cpu_dynamic_df.columns))

gpu_dynamic_df = pd.DataFrame(gpu_dynamic_energy_consumption)
gpu_dynamic_df = gpu_dynamic_df.reindex(columns=sorted(gpu_dynamic_df.columns))

In [44]:
for col in cpu_dynamic_df.columns:
    stat, p = shapiro(cpu_dynamic_df[col])
    alpha = 0.05

    # interpret the result
    if p > alpha:
        print(f'{col} is normally distributed')
    else:
        print(f'{col} is not normally distributed')

fannkuch-redux.exe.lhm.workstationone.all.clang.win32nt is not normally distributed
fannkuch-redux.exe.lhm.workstationone.all.intel-one-api.win32nt is normally distributed
fannkuch-redux.exe.lhm.workstationone.all.min-gw.win32nt is not normally distributed
fannkuch-redux.exe.lhm.workstationone.all.msvc.win32nt is normally distributed
mandelbrot.exe.lhm.workstationone.all.clang.win32nt is normally distributed
mandelbrot.exe.lhm.workstationone.all.intel-one-api.win32nt is not normally distributed
mandelbrot.exe.lhm.workstationone.all.min-gw.win32nt is normally distributed
mandelbrot.exe.lhm.workstationone.all.msvc.win32nt is normally distributed


In [57]:
def Mann(dataFrame):
    for i, col1 in enumerate(dataFrame):
        for j, col2 in enumerate(dataFrame):
            if i < j:  # only compare unique pairs of columns
                stat, p = mannwhitneyu(dataFrame[col1], dataFrame[col2])
                alpha = 0.05

                # interpret the result
                if p > alpha:
                    print(f"{col1} and {col2} have similar medians")
                else:
                    print(f"{col1} and {col2} have significantly different medians")

def Kendall(truthFrame,dataFrame):
    print(f"Kendall's correlation coefficients for {dataFrame.columns[0].split('.')[0]}:")
    for i, col1 in enumerate(dataFrame):
        for j, col2 in enumerate(truthFrame):
            if i < j:  # only compare unique pairs of columns
                corr = dataFrame[col1].corr(dataFrame[col2], method='kendall')
                print(f"{col1} and {col2}: {corr:.3f}")
    print()

In [56]:
fannkuch_df = cpu_dynamic_df.filter(regex='fannkuch-redux')
mandelbrot_df = cpu_dynamic_df.filter(regex='mandelbrot')

Mann(fannkuch_df)
Mann(mandelbrot_df)
Kendall("truth",fannkuch_df)
Kendall("truth", mandelbrot_df)

fannkuch-redux.exe.lhm.workstationone.all.clang.win32nt and fannkuch-redux.exe.lhm.workstationone.all.intel-one-api.win32nt have significantly different medians
fannkuch-redux.exe.lhm.workstationone.all.clang.win32nt and fannkuch-redux.exe.lhm.workstationone.all.min-gw.win32nt have significantly different medians
fannkuch-redux.exe.lhm.workstationone.all.clang.win32nt and fannkuch-redux.exe.lhm.workstationone.all.msvc.win32nt have significantly different medians
fannkuch-redux.exe.lhm.workstationone.all.intel-one-api.win32nt and fannkuch-redux.exe.lhm.workstationone.all.min-gw.win32nt have significantly different medians
fannkuch-redux.exe.lhm.workstationone.all.intel-one-api.win32nt and fannkuch-redux.exe.lhm.workstationone.all.msvc.win32nt have significantly different medians
fannkuch-redux.exe.lhm.workstationone.all.min-gw.win32nt and fannkuch-redux.exe.lhm.workstationone.all.msvc.win32nt have significantly different medians
mandelbrot.exe.lhm.workstationone.all.clang.win32nt and ma